<a href="https://colab.research.google.com/github/GreatLearningAIML1/delhi-may19-batch-Trapti04/blob/master/Statistical_NLP_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from nltk.corpus import stopwords
import spacy

import pandas as pd
import numpy as np
import nltk
from nltk.tokenize.toktok import ToktokTokenizer
import re
from bs4 import BeautifulSoup
# from contractions import CONTRACTION_MAP
import unicodedata

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MultiLabelBinarizer

from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, classification_report,f1_score, accuracy_score, recall_score, precision_score

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
bt_df = pd.read_csv('/content/drive/My Drive/Sequence_models/Statistical files/blogtext.csv' , nrows=5000)

In [4]:
bt_df.head()

,id,gender,age,topic,sign,date,text
0,2059027,male,15,Student,Leo,"14,May,2004","Info has been found (+/- 100 pages,..."
1,2059027,male,15,Student,Leo,"13,May,2004",These are the team members: Drewe...
2,2059027,male,15,Student,Leo,"12,May,2004",In het kader van kernfusie op aarde...
3,2059027,male,15,Student,Leo,"12,May,2004",testing!!! testing!!!
4,3581210,male,33,InvestmentBanking,Aquarius,"11,June,2004",Thanks to Yahoo!'s Toolbar I can ...


In [5]:
bt_df.info() # all are non null columns

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 7 columns):
id        5000 non-null int64
gender    5000 non-null object
age       5000 non-null int64
topic     5000 non-null object
sign      5000 non-null object
date      5000 non-null object
text      5000 non-null object
dtypes: int64(2), object(5)
memory usage: 273.6+ KB


In [6]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [0]:
nlp = spacy.load('en_core_web_sm', parse = False, tag=False, entity=False)
tokenizer = ToktokTokenizer()
stopword_list = nltk.corpus.stopwords.words('english')
#newStopWords = ['None','none','Given','given','give','monday','tuesday','wednesday','thursday','friday','hi','thanks','pm','am','please','task','processing','work','manual']
#stopword_list.extend(newStopWords)

In [0]:
bt_df['gender'] = bt_df['gender'].str.lower()
bt_df['topic'] = bt_df['topic'].str.lower()
bt_df['sign'] = bt_df['sign'].str.lower()

In [0]:
bt_df['labels'] = bt_df[bt_df.columns[1:5]].apply(lambda x: ','.join(x.dropna().astype(str)), axis=1)



In [10]:
bt_df.head()

,id,gender,age,topic,sign,date,text,labels
0,2059027,male,15,student,leo,"14,May,2004","Info has been found (+/- 100 pages,...","male,15,student,leo"
1,2059027,male,15,student,leo,"13,May,2004",These are the team members: Drewe...,"male,15,student,leo"
2,2059027,male,15,student,leo,"12,May,2004",In het kader van kernfusie op aarde...,"male,15,student,leo"
3,2059027,male,15,student,leo,"12,May,2004",testing!!! testing!!!,"male,15,student,leo"
4,3581210,male,33,investmentbanking,aquarius,"11,June,2004",Thanks to Yahoo!'s Toolbar I can ...,"male,33,investmentbanking,aquarius"


In [0]:
def strip_html_tags(text):
    soup = BeautifulSoup(text, "html.parser")
    stripped_text = soup.get_text()
    return stripped_text

In [0]:
def remove_accented_chars(text):
    text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8', 'ignore')
    return text

In [0]:
def remove_special_characters(text):
    #text = re.sub('[^a-zA-z0-9\s]', '', text)
    text = re.sub('[^a-zA-z\s]', ' ', text)
    return text

In [0]:
def lemmatize_text(text):
    text = nlp(text)
    text = ' '.join([word.lemma_ if word.lemma_ != '-PRON-' else word.text for word in text])
    return text

In [0]:
def simple_stemmer(text):
    ps = nltk.porter.PorterStemmer()
    text = ' '.join([ps.stem(word) for word in text.split()])
    return text

In [0]:
def remove_stopwords(text, is_lower_case=False):
    tokens = tokenizer.tokenize(text)
    tokens = [token.strip() for token in tokens]
    if is_lower_case:
        filtered_tokens = [token for token in tokens if token not in stopword_list]
    else:
        filtered_tokens = [token for token in tokens if token.lower() not in stopword_list]
    filtered_text = ' '.join(filtered_tokens)    
    return filtered_text

In [0]:
def normalize_corpus(corpus, html_stripping=True, 
                     accented_char_removal=True, text_lower_case= True, 
                     text_lemmatization= True, text_stemming=True, special_char_removal=True, 
                     stopword_removal=True):
    
    normalized_corpus = []
    # normalize each document in the corpus
    for doc in corpus:
        # strip HTML
        if html_stripping:
            doc = strip_html_tags(doc)
        # remove accented characters
        if accented_char_removal:
            doc = remove_accented_chars(doc)
        # lowercase the text    
        if text_lower_case:
            doc = doc.lower()
        # remove extra newlines
        doc = re.sub(r'[\r|\n|\r\n]+', ' ',doc)
        # insert spaces between special characters to isolate them    
        special_char_pattern = re.compile(r'([{.(-)!}])')
        doc = special_char_pattern.sub(" \\1 ", doc)
        # lemmatize text
        if text_lemmatization:
            doc = lemmatize_text(doc)
        if text_stemming:
            doc = simple_stemmer(doc)
        # remove special characters    
        if special_char_removal:
            doc = remove_special_characters(doc)  
        # remove extra whitespace
        doc = re.sub(' +', ' ', doc)
        # remove stopwords
        if stopword_removal:
            doc = remove_stopwords(doc, is_lower_case=text_lower_case)
        # remove strings starting with zero
        
        normalized_corpus.append(doc)
        
    return normalized_corpus


In [18]:
# pre-process text and store the same
bt_df['processed_text'] = normalize_corpus(bt_df['text'])
#norm_corpus = list(Working_df['clean_text'])

# show a sample process text article
bt_df.iloc[1][['text', 'processed_text']].to_dict()

{'processed_text': 'team member drew van der laag urllink mail ruiyu xie urllink mail bryan aalder urllink mail',
 'text': '           These are the team members:   Drewes van der Laag           urlLink mail  Ruiyu Xie                     urlLink mail  Bryan Aaldering (me)          urlLink mail          '}

In [0]:
bt_df = bt_df.drop(['id','gender', 'age', 'topic', 'sign', 'text', 'date'],axis = 1)

In [20]:
bt_df.head()

,labels,processed_text
0,"male,15,student,leo",info find page mb pdf file wait team leader pr...
1,"male,15,student,leo",team member drew van der laag urllink mail rui...
2,"male,15,student,leo",het kader van kernfusi op aard maak je eigen w...
3,"male,15,student,leo",test test
4,"male,33,investmentbanking,aquarius",thank yahoo toolbar captur url popup mean show...


In [0]:
bt_df.to_csv('BlogText_pre-processed.csv', index=False, encoding='utf-8')

In [0]:
#create X = features and Y = labels

In [0]:
X = bt_df["processed_text"]

In [0]:
Y = bt_df["labels"]

In [0]:
x_train,x_test,y_train,y_test = train_test_split(X, Y, test_size=0.33, random_state=1) 

In [0]:
#creating a Bag of Words Model

In [27]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(min_df = 1,ngram_range = (1,2))
X_train = vectorizer.fit_transform(x_train)
X_test = vectorizer.transform(x_test)
print("X_train shape & sample",X_train.shape)
X_train[0]

X_train shape & sample (3350, 205202)


<1x205202 sparse matrix of type '<class 'numpy.int64'>'
	with 417 stored elements in Compressed Sparse Row format>

In [28]:
vocab = vectorizer.get_feature_names()
print(vocab)


['____', '____ walk', '_topmysteri', '_topmysteri jpg', 'aa', 'aa amaz', 'aaa', 'aaa come', 'aaa someon', 'aaa tow', 'aaa travel', 'aaaaaah', 'aaaaack', 'aaaaahhhh', 'aaaaahhhh heath', 'aaaah', 'aaaah wisdom', 'aaaahh', 'aaahhhh', 'aaahhhh diva', 'aaarrrggghhhhhhhhgggghhhhhh', 'aaarrrggghhhhhhhhgggghhhhhh drop', 'aaja', 'aaja uniti', 'aal', 'aal elimin', 'aal power', 'aal rather', 'aard', 'aard maak', 'aargh', 'aargh tell', 'aaron', 'aaron rowand', 'aaron stephen', 'aaron talladega', 'aaugh', 'aaugh sun', 'ab', 'ab favorit', 'ab mayb', 'ab spi', 'aba', 'aba foundat', 'aba session', 'aba therapi', 'aba well', 'aback', 'aback bit', 'aback take', 'aballabi', 'aballabi want', 'abandon', 'abandon adult', 'abandon although', 'abandon day', 'abandon hi', 'abandon isreal', 'abandon life', 'abandon littl', 'abandon may', 'abba', 'abba gold', 'abba tune', 'abbey', 'abbey jay', 'abbey tri', 'abbi', 'abbi hey', 'abbrevi', 'abbrevi fun', 'abbrevi next', 'abc', 'abc famili', 'abc keep', 'abc last', 

In [0]:
cv_matrix = X_train.toarray()

In [30]:
# show document feature vectors
pd.DataFrame(cv_matrix, columns=vocab)

,____,____ walk,_topmysteri,_topmysteri jpg,aa,aa amaz,aaa,aaa come,aaa someon,aaa tow,aaa travel,aaaaaah,aaaaack,aaaaahhhh,aaaaahhhh heath,aaaah,aaaah wisdom,aaaahh,aaahhhh,aaahhhh diva,aaarrrggghhhhhhhhgggghhhhhh,aaarrrggghhhhhhhhgggghhhhhh drop,aaja,aaja uniti,aal,aal elimin,aal power,aal rather,aard,aard maak,aargh,aargh tell,aaron,aaron rowand,aaron stephen,aaron talladega,aaugh,aaugh sun,ab,ab favorit,...,zone natur,zone origin,zone particular,zone primari,zone simpli,zone somehow,zone sometim,zone storm,zone today,zoni,zoni well,zoo,zoo downtown,zoo exhibit,zoo roster,zoo york,zookz,zoom,zoom first,zoom much,zoom one,zoom past,zua,zua watch,zuluchim,zuluchim hey,zuluchim oh,zuluchim pharsalus,zuluchim uh,zuluchim veri,zun,zun charl,zun person,zurich,zurich switzerland,zza,zza dong,zzzz,zzzz get,zzzzzzz
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3345,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3346,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3347,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3348,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [0]:
#Create a dictionary to get the count of every label i.e. the key will be label name and value will be the total count of the label.

In [32]:
vectorizer_labels = CountVectorizer(min_df = 2,ngram_range = (1,1))
labels_vector = vectorizer_labels.fit_transform(Y)
vectorizer_labels.vocabulary_

{'14': 0,
 '15': 1,
 '16': 2,
 '17': 3,
 '23': 4,
 '24': 5,
 '25': 6,
 '26': 7,
 '27': 8,
 '33': 9,
 '34': 10,
 '35': 11,
 '36': 12,
 '37': 13,
 '39': 14,
 '41': 15,
 '42': 16,
 '44': 17,
 '45': 18,
 '46': 19,
 'accounting': 20,
 'aquarius': 21,
 'aries': 22,
 'arts': 23,
 'automotive': 24,
 'banking': 25,
 'businessservices': 26,
 'cancer': 27,
 'capricorn': 28,
 'communications': 29,
 'consulting': 30,
 'education': 31,
 'engineering': 32,
 'female': 33,
 'gemini': 34,
 'indunk': 35,
 'internet': 36,
 'investmentbanking': 37,
 'law': 38,
 'leo': 39,
 'libra': 40,
 'libraries': 41,
 'male': 42,
 'media': 43,
 'museums': 44,
 'non': 45,
 'pisces': 46,
 'profit': 47,
 'recreation': 48,
 'religion': 49,
 'sagittarius': 50,
 'science': 51,
 'scorpio': 52,
 'sports': 53,
 'student': 54,
 'taurus': 55,
 'technology': 56,
 'virgo': 57}

In [33]:
# extract the unique label classes

label_classes = []  
for key in vectorizer_labels.vocabulary_.keys():
    label_classes.append(key)
    
print(sorted(label_classes))

['14', '15', '16', '17', '23', '24', '25', '26', '27', '33', '34', '35', '36', '37', '39', '41', '42', '44', '45', '46', 'accounting', 'aquarius', 'aries', 'arts', 'automotive', 'banking', 'businessservices', 'cancer', 'capricorn', 'communications', 'consulting', 'education', 'engineering', 'female', 'gemini', 'indunk', 'internet', 'investmentbanking', 'law', 'leo', 'libra', 'libraries', 'male', 'media', 'museums', 'non', 'pisces', 'profit', 'recreation', 'religion', 'sagittarius', 'science', 'scorpio', 'sports', 'student', 'taurus', 'technology', 'virgo']


In [0]:
# initial the multilabel binarizer with all possible values

In [0]:
mlb = MultiLabelBinarizer(classes = label_classes)

In [36]:
# Converting entire se of labels into format required by mlb
labels = [["".join(re.findall("\w",f)) for f in lst] for lst in [s.split(",") for s in Y]]
labels[30]

['male', '33', 'investmentbanking', 'aquarius']

In [37]:
labels_trans = mlb.fit(labels) # transforming entire set of lables
labels_trans

MultiLabelBinarizer(classes=['male', '15', 'student', 'leo', '33',
                             'investmentbanking', 'aquarius', 'female', '14',
                             'indunk', 'aries', '25', 'capricorn', '17',
                             'gemini', '23', 'non', 'profit', 'cancer',
                             'banking', '37', 'sagittarius', '26', '24',
                             'scorpio', '27', 'education', '45', 'engineering',
                             'libra', ...],
                    sparse_output=False)

In [38]:
#Convert Y_train into a format as required by mlb 
y_train = [["".join(re.findall("\w",f)) for f in lst] for lst in [s.split(",") for s in y_train]]
y_train[30]

['female', '27', 'education', 'aquarius']

In [39]:
Y_train = mlb.transform(y_train) # transforming Train lables using mlb which is trained on all possible unnique labels on entire data set
Y_train[30]

/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_label.py:987: UserWarning: unknown class(es) ['communicationsmedia', 'museumslibraries', 'nonprofit', 'sportsrecreation'] will be ignored
  .format(sorted(unknown, key=str)))


array([0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [40]:
#Convert y_test into a format as required by mlb 
y_test = [["".join(re.findall("\w",f)) for f in lst] for lst in [s.split(",") for s in y_test]]
Y_test = mlb.transform(y_test) # transforming test labels.
print(Y_test[30])

[1 0 1 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]


/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_label.py:987: UserWarning: unknown class(es) ['communicationsmedia', 'museumslibraries', 'nonprofit', 'sportsrecreation'] will be ignored
  .format(sorted(unknown, key=str)))


In [41]:
len(mlb.classes_)

58

In [42]:
mlb.classes_

array(['male', '15', 'student', 'leo', '33', 'investmentbanking',
       'aquarius', 'female', '14', 'indunk', 'aries', '25', 'capricorn',
       '17', 'gemini', '23', 'non', 'profit', 'cancer', 'banking', '37',
       'sagittarius', '26', '24', 'scorpio', '27', 'education', '45',
       'engineering', 'libra', 'science', '34', '41', 'communications',
       'media', 'businessservices', 'sports', 'recreation', 'virgo',
       'taurus', 'arts', 'pisces', '44', '16', 'internet', 'museums',
       'libraries', 'accounting', '39', '35', 'technology', '36', 'law',
       '46', 'consulting', 'automotive', '42', 'religion'], dtype=object)

In [43]:
y_test[10]

['male', '35', 'technology', 'aries']

In [44]:
Y_test[10]

array([1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0])

In [45]:
Y_test.shape

(1650, 58)

In [0]:
clf = LogisticRegression(solver = 'lbfgs',max_iter = 1000)  # initiating the classifier

clf = OneVsRestClassifier(clf)

In [47]:
clf.fit(X_train,Y_train) # Fitting on  train data

/usr/local/lib/python3.6/dist-packages/sklearn/multiclass.py:75: UserWarning: Label not 16 is present in all training examples.
  str(classes[c]))
/usr/local/lib/python3.6/dist-packages/sklearn/multiclass.py:75: UserWarning: Label not 17 is present in all training examples.
  str(classes[c]))
/usr/local/lib/python3.6/dist-packages/sklearn/multiclass.py:75: UserWarning: Label not 33 is present in all training examples.
  str(classes[c]))
/usr/local/lib/python3.6/dist-packages/sklearn/multiclass.py:75: UserWarning: Label not 34 is present in all training examples.
  str(classes[c]))
/usr/local/lib/python3.6/dist-packages/sklearn/multiclass.py:75: UserWarning: Label not 36 is present in all training examples.
  str(classes[c]))
/usr/local/lib/python3.6/dist-packages/sklearn/multiclass.py:75: UserWarning: Label not 37 is present in all training examples.
  str(classes[c]))
/usr/local/lib/python3.6/dist-packages/sklearn/multiclass.py:75: UserWarning: Label not 45 is present in all training 

OneVsRestClassifier(estimator=LogisticRegression(C=1.0, class_weight=None,
                                                 dual=False, fit_intercept=True,
                                                 intercept_scaling=1,
                                                 l1_ratio=None, max_iter=1000,
                                                 multi_class='auto',
                                                 n_jobs=None, penalty='l2',
                                                 random_state=None,
                                                 solver='lbfgs', tol=0.0001,
                                                 verbose=0, warm_start=False),
                    n_jobs=None)

In [48]:
print("Train Accuracy:",clf.score(X_train,Y_train))

Train Accuracy: 0.9767164179104477


In [0]:
Y_pred = clf.predict(X_test)

In [50]:
print("Test Accuracy:" + str(accuracy_score(Y_test, Y_pred)))
print("F1: " + str(f1_score(Y_test, Y_pred, average='micro')))
print("F1_macro: " + str(f1_score(Y_test, Y_pred, average='macro')))
print("Precision: " + str(precision_score(Y_test, Y_pred, average='micro')))
print("Precision_macro: " + str(precision_score(Y_test, Y_pred, average='macro')))
print("Recall: " + str(recall_score(Y_test, Y_pred, average='micro')))
print("Recall_macro: " + str(recall_score(Y_test, Y_pred, average='macro')))

Test Accuracy:0.5139393939393939
F1: 0.7294157265532918
F1_macro: 0.23742546752310176
Precision: 0.8118440779610195
Precision_macro: 0.39715662180014094
Recall: 0.6621828187098746
Recall_macro: 0.19287092838030004


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1515: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  average, "true nor predicted", 'F-score is', len(true_sum)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# Model analysis

Clearly the model is overfit as can be observed by the train v/s test accuracy metric difference. To improve the accuracy lemmization has not improved - result remained same.

A typical multi-class classification problem, we need to categorize each sample into 1 of N different classes, and the precision and recall is calculated over the set of 1 to N classes.

for e,g precision will be the no. of instances of the **correctly** classified 'class' over the total no. of that 'class' classifications by model.

Similar the recall values are calculated over eth N classes.

The sklearn.metrics.classification_report is also a handy tool to give the classification  scores for a  multi class classification.

F1-score will always be somewhere in between precision and mean.
F1-score = 2 × (precision × recall)/(precision + recall). It behaves differently then arithematic mean and generally - the F1-score gives a larger weight to lower numbers and hence pulls the score more towrads teh lower numbers.
By "macro" FI / precision / recall we mean that the calaclation is performed over the global n*m set and  usually in datascience it is the default metric used.


In [0]:
Y_pred_inv = mlb.inverse_transform(Y_pred)   # inverse transforming predited label data
Y_test_inv =  mlb.inverse_transform(Y_test) # inverse transforming original test label data

In [52]:
print("Example 1 - predicted :",Y_pred_inv[0])
print("Example 1 - Actual :",Y_test_inv[0])
print("Example 1 - Actual_before mlb transformation :",y_test[0])

Example 1 - predicted : ('male', 'aries', '35', 'technology')
Example 1 - Actual : ('male', 'aries', '35', 'technology')
Example 1 - Actual_before mlb transformation : ['male', '35', 'technology', 'aries']


In [53]:
print("Example 2 - predicted :",Y_pred_inv[50])
print("Example 2 - Actual :",Y_test_inv[50])
print("Example 2 - Actual_before mlb transformation :",y_test[50])

Example 2 - predicted : ('male', 'indunk')
Example 2 - Actual : ('male', 'indunk', '25', 'taurus')
Example 2 - Actual_before mlb transformation : ['male', '25', 'indunk', 'taurus']


In [54]:
print("Example 3 - predicted :",Y_pred_inv[500])
print("Example 3 - Actual :",Y_test_inv[500])
print("Example 3 - Actual_before mlb transformation :",y_test[500])

Example 3 - predicted : ('male', 'aries', '35', 'technology')
Example 3 - Actual : ('male', 'aries', '35', 'technology')
Example 3 - Actual_before mlb transformation : ['male', '35', 'technology', 'aries']


In [58]:
print("Example 4 - predicted :",Y_pred_inv[602])
print("Example 4 - Actual :",Y_test_inv[602])
print("Example 4 - Actual_before mlb transformation :",y_test[602])

Example 4 - predicted : ('male', 'aries', '35', 'technology')
Example 4 - Actual : ('aquarius', 'female', 'indunk', '37')
Example 4 - Actual_before mlb transformation : ['female', '37', 'indunk', 'aquarius']


In [56]:
print("Example 5 - predicted :",Y_pred_inv[800])
print("Example 5 - Actual :",Y_test_inv[800])
print("Example 5 - Actual_before mlb transformation :",y_test[800])

Example 5 - predicted : ('male', 'aries', '35', 'technology')
Example 5 - Actual : ('male', 'aries', '35', 'technology')
Example 5 - Actual_before mlb transformation : ['male', '35', 'technology', 'aries']


# learnings

The model is overfotting
Lemmization and stemming the features have little impact on accuracy
As the no. of records are training increases the model gets more overfit. Starting with the  300 dataset with 57% test accuracy the accuracy decreased to 51%  for 500 records.
The min_df parameter in countervectorizer has no impact on model accuracy either.

More exploration on how the  model can be improved wi th hyperparameerization further or feature engineering.